In [30]:
# Import our dependencies
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [31]:
mlb_df = pd.read_csv("Resources/mlb_complete_8years.csv")
mlb_df.drop(columns=['Unnamed: 0','W','L','W-L%'], inplace=True)
mlb_df.head()

,Tm,BatAge,R/G,AB,R_x,H_x,2B,3B,HR_x,RBI,...,BF,LOB_y,DefEff,Ch,PO,A,E,DP,post_season,Year
0,Arizona Diamondbacks_2022,26.5,4.33,5351,702,1232,262,24,173,658,...,6065,1051,0.704,5746,4290,1370,86,134,False,2022
1,Atlanta Braves_2022,27.5,4.87,5509,789,1394,298,11,243,753,...,6031,1101,0.701,5803,4344,1382,77,110,True,2022
2,Baltimore Orioles_2022,27.0,4.16,5429,674,1281,275,25,171,639,...,6058,1092,0.690,5920,4300,1529,91,151,False,2022
3,Boston Red Sox_2022,28.8,4.54,5539,735,1427,352,12,155,704,...,6167,1109,0.683,5825,4293,1447,85,134,False,2022
4,Chicago Cubs_2022,27.9,4.06,5425,657,1293,265,31,159,620,...,6162,1130,0.697,5880,4331,1453,96,139,False,2022


In [32]:
features_df = mlb_df.drop(columns=['Tm','post_season','Year'])
target_df = mlb_df['post_season'].astype(int)

In [33]:
features_df.shape

(239, 36)

In [34]:
from sklearn.decomposition import PCA
pca = PCA(n_components=6)
mlb_pca = pca.fit_transform(features_df)
mlb_pca[:5]

array([[-660.17736599,  -89.1351583 ,  -72.26480806,  -37.87896619,
         -14.26309992,  -58.67955494],
       [-832.57981515,  270.17523125, -146.96397207,  -82.9496826 ,
         107.10382511,   17.79973334],
       [-824.01380981, -114.31109402,  111.99250575,  -61.44781827,
           2.05651818,  -58.03350334],
       [-925.01527735,  -23.59718876,  -90.44901649,   38.13238635,
        -120.60851713,   11.01698617],
       [-886.91354355,  -51.08908062,  -59.97164729, -148.96036431,
          11.29352921,  -69.77139828]])

In [35]:
mlb_pca.shape

(239, 6)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(mlb_pca, target_df, random_state=78)
X_scaler = skl.preprocessing.StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [37]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=6))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [38]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json


In [39]:
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

INFO:tensorflow:Oracle triggered exit


In [40]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 3,
 'num_layers': 2,
 'units_0': 9,
 'units_1': 7,
 'units_2': 5,
 'units_3': 7,
 'units_4': 1,
 'units_5': 3,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [41]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

ValueError: Received incompatible tensor with shape (36, 3) when attempting to restore variable with shape (6, 3) and name dense/kernel:0.